In [1]:
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

2023-05-31 23:11:35.594761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 23:11:38.071948: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ncol = 384
nlev = 60
tlen = 2543616

In [56]:
# Convert to pressure
def convert_to_Pressure(hyai,hybi,PS,P0):
    Dimension1=hyai.shape
    Dimension2=PS.shape
    Pressure = np.zeros([Dimension2[1],Dimension1[0]])
    
    for i in range(Dimension2[1]):
        Pressure[i,:] = (P0*hyai[:] + PS[0,i]*hybi[:])
    return Pressure

In [4]:
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')
lon = ne4_grid_info.lon.values
lat = ne4_grid_info.lat.values
area = ne4_grid_info.area.values
hyai = ne4_grid_info.hyai.values
hybi = ne4_grid_info.hybi.values
PS = ne4_grid_info.PS.values
P0 = ne4_grid_info.P0.values

In [57]:
Pre = convert_to_Pressure(hyai,hybi,PS,P0)

In [6]:
dPre = Pre[:,1:nlev+1] - Pre[:,0:nlev] 

In [7]:
mli_mean = xr.open_dataset('./norm_factors/mli_mean.nc')
mli_min = xr.open_dataset('./norm_factors/mli_min.nc')
mli_max = xr.open_dataset('./norm_factors/mli_max.nc')
mlo_scale = xr.open_dataset('./norm_factors/mlo_scale.nc')
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')

def load_nc_dir_with_generator_test(filelist:list, latlim=[-999,999], lonlim=[-999,999]):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]
            ds = ds.merge(ne4_grid_info[['lat','lon']])
            ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
            ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            dso = dso.merge(ne4_grid_info[['lat','lon']])
            dso = dso.where((dso['lat']>latlim[0])*(dso['lat']<latlim[1]),drop=True)
            dso = dso.where((dso['lon']>lonlim[0])*(dso['lon']<lonlim[1]),drop=True)
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalizatoin, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

In [10]:
# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

In [11]:
# validation dataset for HPO
stride_sample = 5 
#f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc')
#f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-1[012]-*-*.nc')
f_mli3 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0009-01-*-*.nc')
#f_mli_val = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli = f_mli3
print(f'#files: {len(f_mli)}')

#files: 2232


In [12]:
tds_test = load_nc_dir_with_generator_test(f_mli)
work = list(tds_test.as_numpy_iterator())
x_true_n = np.concatenate([ work[k][0] for k in range(len(work)) ])
y_true_n = np.concatenate([ work[k][1] for k in range(len(work)) ])

2023-05-31 23:14:06.950398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [13]:
# This function turns 20 minute increments into daily averages
# (384)*ndays*72 = Dim[0]
# Output dim: (384)*ndays
# Griffin's original version is based on regrided data
# This version is based on ne4pg2 grid (384 columns)
# This change allow me to use Sungduk's code directly
# since he generaged the min/max/scale already
def average_data_Griffin_LiranEdited(reconstructed_targets,reconstructed_features):
    Dim = reconstructed_targets.shape
    x = 384
    tnum = 72
    t = Dim[0]
    z = Dim[1]
    ndays = int(t/(tnum*x))
    
    target_days = np.zeros(shape=(x*ndays,tnum, z))
    feat_days = np.zeros(shape=(x*ndays,tnum, z))
    day_array_targ = np.zeros(shape=(x,tnum,ndays, z))
    day_array_feat = np.zeros(shape=(x,tnum,ndays, z))
    #print(day_array_feat.shape)
    ncol_count = 0
    tstep_count = 0
    day_count = 0
    
    for i in range(t):
        temp_targ = reconstructed_targets[i, :]
        day_array_targ[ncol_count,tstep_count,day_count, :] = temp_targ
        temp_feat = reconstructed_features[i, :]
        day_array_feat[ncol_count,tstep_count,day_count,:] = temp_feat
        
        if (ncol_count == x-1):
            ncol_count = 0
            tstep_count = tstep_count+1
        else:
            ncol_count = ncol_count+1
        
        if (tstep_count == tnum):
            tstep_count = 0
            day_count = day_count+1   
            
            
    day_array_targ_out = np.nanmean(day_array_targ, axis = 1)
    day_array_feat_out = np.nanmean(day_array_feat, axis = 1)
    
    return day_array_targ_out,day_array_feat_out

In [14]:
import os
import netCDF4 as nc
folder_path = "/pscratch/sd/h/heroplr/step2_retrain/backup_phase-4_retrained_models"  # Replace with the actual folder path
out_folder = "/pscratch/sd/h/heroplr/R2_analysis_all/"
Dim_true = x_true_n.shape
# Loop through all files in the folder
numday = int(Dim_true[0]/ncol/72)
filename="step2_lot-101_trial_0028.best.h5"
file_path = os.path.join(folder_path, filename)

In [15]:
model = keras.models.load_model(file_path,compile=False)
y_pred = model(x_true_n)
T_tend_true = y_true_n[:,:60]
T_pred_true = y_pred[:,:60]
Q_tend_true = y_true_n[:,60:120]
Q_pred_true =y_pred[:,60:120]

In [68]:
mweightpre    =  dPre/9.8  # dP/g

In [18]:
T_tend_true2,T_pred_true2 = average_data_Griffin_LiranEdited(T_tend_true,T_pred_true)
Q_tend_true2,Q_pred_true2 = average_data_Griffin_LiranEdited(Q_tend_true,Q_pred_true)

In [63]:
T_tend_true2.shape

(384, 31, 60)

In [64]:
T_tend_true_daymean = np.mean(T_tend_true2,axis=1)
T_pred_true_daymean = np.mean(T_pred_true2,axis=1)
Q_tend_true_daymean = np.mean(Q_tend_true2,axis=1)
Q_pred_true_daymean = np.mean(Q_pred_true2,axis=1)

In [69]:
cp_dry_o   = 1006
latent_heat_vaporization = 2.501e6 
T_tend_true_mweight    = T_tend_true_daymean*mweightpre*cp_dry_o
T_pred_true_mweight    = T_pred_true_daymean*mweightpre*cp_dry_o
Q_tend_true_mweight    = Q_tend_true_daymean*mweightpre*latent_heat_vaporization
Q_pred_true_mweight    = Q_pred_true_daymean*mweightpre*latent_heat_vaporization

In [70]:
# Create a new NetCDF file
filename = file_path[-31:]+"_mwv9.nc"
file_path_out = os.path.join(out_folder, filename)
        
ncfile = nc.Dataset(file_path_out, "w", format="NETCDF4")

# Define the dimensions
time_dim = ncfile.createDimension("time", None)  # Unlimited dimension
lat_dim = ncfile.createDimension("ncol", ncol)
lon_dim = ncfile.createDimension("nlev", nlev)
# Create variables
time_var = ncfile.createVariable("time", "f8", ("time",))
lon_var = ncfile.createVariable("lon", "f4", ("ncol",))
lat_var = ncfile.createVariable("lat", "f4", ("ncol",))
data_var3 = ncfile.createVariable("mwT_tend", "f8", ("nlev","ncol"))
data_var4 = ncfile.createVariable("mwT_pred", "f8", ("nlev","ncol"))
data_var5 = ncfile.createVariable("mwQ_tend", "f8", ("nlev","ncol"))
data_var6 = ncfile.createVariable("mwQ_pred", "f8", ("nlev","ncol"))

# Assign values to variables
time_var[:] = [1]  # Example time values
lon_var[:] = lon    # Example latitude values
lat_var[:] = lat  # Example longitude values

data_var3[:,:] = np.transpose(T_tend_true_mweight)
data_var4[:,:] = np.transpose(T_pred_true_mweight)
data_var5[:,:] = np.transpose(Q_tend_true_mweight)
data_var6[:,:] = np.transpose(Q_pred_true_mweight)
# Add global attributes
ncfile.description = "R2"
ncfile.history = "Created by Liran"

# Close the NetCDF file
ncfile.close()
    

In [71]:
Q_pred_true_daymean[40,:]

array([ 6.10942800e-07,  8.18767238e-06,  5.64713680e-07, -1.02400747e-04,
        1.64642161e-06, -7.93141568e-08,  2.89449495e-05, -8.76644755e-08,
        5.66618703e-07, -4.81629807e-06,  2.96376775e-06, -1.76648857e-07,
        4.54410113e-06, -1.92427276e-05, -2.81681329e-07,  7.89377296e-06,
       -3.10016791e-05,  6.31907780e-06, -8.92952857e-06, -9.71249335e-05,
        1.22069544e-04, -3.22580278e-04, -4.85072855e-05,  2.59634970e-04,
       -2.32376288e-04, -1.25164825e-04, -5.97484314e-04, -2.20339302e-03,
       -2.05172243e-03, -3.83600999e-04,  7.75983625e-04,  1.01691550e-03,
        1.00773819e-03, -2.14840769e-04, -1.84841889e-03, -1.03585992e-04,
        3.31007305e-03,  8.03046947e-03,  5.24557703e-03,  4.54257330e-03,
        2.78044845e-03,  1.04446673e-03,  5.37678384e-03,  5.08641572e-03,
        5.16825489e-03, -3.20751930e-03, -1.67627475e-03,  1.11209705e-03,
       -2.81822137e-03,  1.64776450e-03, -3.93438148e-03, -4.41322629e-03,
       -1.62150669e-03, -

In [72]:
Q_tend_true_daymean[40,:]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        9.46714821e-10,  2.55652626e-09, -2.28443622e-08, -4.69418046e-07,
        4.60578867e-07,  1.38002524e-06,  6.56194788e-07,  3.80375574e-07,
       -6.82573169e-06, -4.40816635e-07,  2.24007211e-05, -2.80675797e-05,
       -4.49974514e-04, -9.41926319e-04, -1.13992738e-03, -1.70298687e-03,
       -2.49279744e-03, -1.76471924e-03, -1.05997026e-03, -1.03575038e-04,
        1.27006437e-03,  2.04081967e-03,  1.99418642e-03,  6.76904743e-04,
        2.09315280e-04,  1.01291297e-03,  1.76391738e-03,  2.57989935e-03,
        3.28014197e-03, -1.23524340e-03, -4.73093302e-04, -1.43850475e-03,
       -3.34069199e-04, -7.61933672e-05,  2.19315823e-04,  1.92231055e-04,
        4.94740505e-04,  3.41795856e-04,  6.03989943e-04,  1.18938714e-03,
        1.95040788e-03,  

In [73]:
mweightpre[40,:]

array([  0.45850491,   0.82232176,   1.45961657,   2.5470638 ,
         4.33243674,   7.11673875,  11.19798629,  16.79495649,
        24.00922601,  32.87683103,  43.47492677,  55.96055633,
        70.44699656,  86.76827172, 104.31814462, 122.15229792,
       139.37007078, 155.53664753, 170.7987482 , 185.55189543,
       199.88749742, 213.24544761, 224.24655082, 227.60972153,
       232.55147614, 235.11448309, 236.81416986, 239.2171681 ,
       243.31632578, 249.29376316, 256.71290265, 264.91998042,
       273.36410152, 281.65435346, 289.36833477, 295.795698  ,
       299.85382964, 300.31913073, 296.3183605 , 287.8254305 ,
       275.84172516, 262.08337395, 248.29607   , 235.56668456,
       224.01265156, 212.9960277 , 201.70041515, 189.72433001,
       177.37508258, 165.54838505, 155.31754361, 147.49007978,
       142.35485593, 139.69149761, 138.96352974, 139.55455148,
       140.93875175, 142.75027788, 144.7745171 , 146.90497813])

In [74]:
Pre[40,:]

array([5.58810705e+00, 1.00814552e+01, 1.81402085e+01, 3.24444509e+01,
       5.74056761e+01, 9.98635562e+01, 1.69607596e+02, 2.79347861e+02,
       4.43938435e+02, 6.79228850e+02, 1.00142179e+03, 1.42747608e+03,
       1.97588953e+03, 2.66627009e+03, 3.51659916e+03, 4.53891697e+03,
       5.73600949e+03, 7.10183619e+03, 8.62609533e+03, 1.02999231e+04,
       1.21183316e+04, 1.40772291e+04, 1.61670345e+04, 1.83646507e+04,
       2.05952260e+04, 2.28742304e+04, 2.51783524e+04, 2.74991312e+04,
       2.98434595e+04, 3.22279595e+04, 3.46710384e+04, 3.71868248e+04,
       3.97830406e+04, 4.24620088e+04, 4.52222215e+04, 4.80580311e+04,
       5.09568290e+04, 5.38953965e+04, 5.68385240e+04, 5.97424439e+04,
       6.25631332e+04, 6.52663821e+04, 6.78347991e+04, 7.02681006e+04,
       7.25766541e+04, 7.47719781e+04, 7.68593392e+04, 7.88360032e+04,
       8.06953017e+04, 8.24335775e+04, 8.40559517e+04, 8.55780636e+04,
       8.70234664e+04, 8.84185440e+04, 8.97875206e+04, 9.11493632e+04,
      

In [75]:
dPre[40,:]

array([   4.49334817,    8.05875326,   14.30424243,   24.96122521,
         42.45788005,   69.74403971,  109.74026562,  164.59057361,
        235.29041485,  322.19294405,  426.05428239,  548.41345205,
        690.3805663 ,  850.32906287, 1022.31781732, 1197.09251962,
       1365.82669363, 1524.25914584, 1673.82773233, 1818.40857522,
       1958.89747468, 2089.80538655, 2197.61619805, 2230.57527096,
       2279.00446613, 2304.12193427, 2320.77886458, 2344.32824742,
       2384.49999267, 2443.07887894, 2515.78644598, 2596.21580808,
       2678.96819489, 2760.21266389, 2835.80968071, 2898.79784042,
       2938.56753046, 2943.12748119, 2903.91993285, 2820.68921891,
       2703.24890659, 2568.41706473, 2433.30148599, 2308.55350874,
       2195.32398533, 2087.36107146, 1976.6640685 , 1859.29843412,
       1738.27580931, 1622.37417351, 1522.11192742, 1445.40278183,
       1395.07758815, 1368.97667654, 1361.84259141, 1367.63460452,
       1381.19976716, 1398.95272319, 1418.79026761, 1439.66878

In [76]:
Q_tend_true_daymean[40,:]*mweightpre[40,:]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        6.66932157e-08,  2.21825366e-07, -2.38308148e-06, -5.73404930e-05,
        6.41909093e-05,  2.14644499e-04,  1.12077248e-04,  7.05794087e-05,
       -1.36437843e-03, -9.40021407e-05,  5.02328445e-03, -6.38845400e-03,
       -1.04642237e-01, -2.21460520e-01, -2.69950957e-01, -4.07383697e-01,
       -6.06538315e-01, -4.39933500e-01, -2.72108041e-01, -2.74390969e-02,
        3.47190007e-01,  5.74805745e-01,  5.77054404e-01,  2.00225511e-01,
        6.27639884e-02,  3.04197142e-01,  5.22681106e-01,  7.42560641e-01,
        9.04800020e-01, -3.23736757e-01, -1.17467208e-01, -3.38863796e-01,
       -7.48357270e-02, -1.62288846e-02,  4.42360926e-02,  3.64709081e-02,
        8.77546379e-02,  5.65837520e-02,  9.38102343e-02,  1.75422804e-01,
        2.77650032e-01,  